In [1]:
%display latex

# Black Hole Rendering
Following work by [Florentin Jaffredo](https://sagemanifolds.obspm.fr/gallery.html), relying on SageMath's geodesic integrator.

The approach is to use null-geodesics around an [Eddington-Finkelstein](https://en.wikipedia.org/wiki/Eddington%E2%80%93Finkelstein_coordinates) black hole, as the coordinates behave desireably when computing null-geodesics around the event horizon.

We will assume spherical symmetry to reduce the number of geodesic calculations required.

The spacetime we work in will be declared as

In [2]:
# manifold
M = Manifold(4, "M", structure="Lorentzian")
# chart
C.<t, r, theta, phi> = M.chart(r"t r:(1,+oo) theta:(0,pi):\theta phi:\phi")

C.coord_range()

t: (-oo, +oo); r: (1, +oo); theta: (0, pi); phi: (-oo, +oo)

with a metric $g$ for central mass $m$

In [3]:
m = var('m')
g = M.metric()

# basis
g[0,0] = -(1 - 2*m/r)
g[0,1] = 2*m/r
g[1,1] = 1 + 2*m/r
g[2,2] = r^2
g[3,3] = (r*sin(theta))^2

g[:]

[       2*m/r - 1            2*m/r                0                0]
[           2*m/r        2*m/r + 1                0                0]
[               0                0              r^2                0]
[               0                0                0 r^2*sin(theta)^2]

We also require an Euclidean space to present and plot results in, for which we define a map to our manifold:

In [4]:
E.<x, y, z> = EuclideanSpace()

# transform map
phimap = M.diff_map(
    E,
    [
        r*sin(theta)*cos(phi),
        r*sin(theta)*sin(phi),
        r*cos(theta)
    ]
)

phimap.display()

M --> E^3
   (t, r, theta, phi) |--> (x, y, z) = (r*cos(phi)*sin(theta), r*sin(phi)*sin(theta), r*cos(theta))

## Plotting geodesics
We will use the `integrated_geodesic` functionality of [SageManifold](https://doc.sagemath.org/html/en/reference/manifolds/sage/manifolds/differentiable/integrated_curve.html)

In [5]:
import multiprocessing
from ipywidgets import FloatProgress
from IPython.display import display

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [6]:
BATCH_PER_CPU = 3
NUM_GEOD = 1000
N_CPU = 8

Need to find starting velocities: our conditions are
- past-oriented
- light-like
- pointing towards the blackhole with increasing angle


Spatial components are determined, so we adjust the time component to ensure light-like. For some inital point $p$, we want initial 4-velocity $v$ constrained by the time coordinate $dt$:

In [7]:
dt, y, r0 = var("dt, y, r0")
p = M((0, r0, pi/2, 0)) # initial point
Tp = M.tangent_space(p)
v = Tp((dt, -1, 0, y))

# norm:
g.at(p)(v, v)

r0^2*y^2 + dt^2*(2*m - r0)/r0 - 4*dt*m/r0 + (2*m + r0)/r0

Find $dt$ such that $g=0$, thus

In [8]:
dt_constraint = g.at(p)(v, v).solve(dt)
dt_constraint

[dt == -(sqrt(-(2*m*r0 - r0^2)*y^2 + 1)*r0 - 2*m)/(2*m - r0), dt == (sqrt(-(2*m*r0 - r0^2)*y^2 + 1)*r0 + 2*m)/(2*m - r0)]

Two solutions represent past and future orientation.

Define geodesic problem, including checking for chart boundary:

In [9]:
tau = var("tau", latex_name=r"\tau")

# arbitrary initial vectors required
# seeds the geodesic curve and sets state appropriate
p = M((0, 5, pi/2, 0))
Tp = M.tangent_space(p)
v = Tp((0, 5, 0, 0))
v = v / sqrt(-g.at(p)(v, v)) # normalize

# init curve
curve = M.integrated_geodesic(
    g,
    (tau, 0, 200), # parameterization
    v,
    across_charts=True
)

Prepare multiprocessing steps by packing arguments into tuples

In [10]:
args = []
start = 0

for chunk in chunks(
        range(NUM_GEOD), NUM_GEOD//(BATCH_PER_CPU * N_CPU)
    ):
    args.append((loads(curve.dumps()), start, len(chunk)))
    start += len(chunk)

In [11]:
print("- Final argument tuple : ", args[-1])
print("- Number of argument batches: ", len(args))

- Final argument tuple :  (Integrated geodesic in the 4-dimensional Lorentzian manifold M, 984, 16)
- Number of argument batches:  25


The function used to calculate the geodesic batches:

In [12]:
def calc_geodesic(args):
    """ Multiproc function for calculating `num` geodesics at `n0` """
    curve, n0, num = args
    res = {}
    r = 100
    p = M([0, r, pi/2, 0])
    Tp = M.tangent_space(p)
    
    for i in range(n0, n0+num):
        
        # starting vector
        dy = i * 0.0006 / NUM_GEOD # magic number for scanning angle increase
        v = Tp(
            [
                dt_constraint[0].rhs()(r0=r, y=dy, m=2).n(), # numeric
                -1, 0, dy
            ]
        )
        
        # overwrite starting vector in curve
        curve._initial_tangent_vector = v
        
        # integrate
        curve.solve_across_charts(
            step=0.2, 
            parameters_values={m:2}
        )
        
        # copy and clear solutions
        res[i] = (p.coord(), curve._solutions.copy())
        curve._solutions.clear()
    
    return res 
    

Now we execute our code. We store the solutions in `geodesics`, and use the functionality of `Pool` to handle system resources:

In [13]:
geodesics = {}
pool = multiprocessing.Pool(N_CPU)

# display bar
%display plain
progress = FloatProgress(min=0, max=NUM_GEOD)
display(progress)

for i, result in enumerate(
        pool.imap_unordered(calc_geodesic, args)
    ):
    # imap_unordered executes and halts
    progress.value += len(result)
    geodesics.update(result)
    
pool.close()
pool.join()

%display latex

FloatProgress(value=0.0, max=1000.0)

### Visualizing result:
Plotting a subset of the geodesics as curves in spacetime:

In [16]:
bh = sage.plot.plot3d.shapes.Sphere(
    4, 
    color="black", 
    viewer="threejs", 
    spect_ratio=[1,1,1]
)

for i in range(0, NUM_GEOD, 5 * NUM_GEOD / 100):
    curve._solutions = geodesics[i][1]
    
    # interpolate solutions
    interp = curve.interpolate()
    
    bh += curve.plot_integrated(
        mapping=phimap,
        color=["red"],
        thickness=2,
        plot_points=200,
        label_axes=False,
        across_charts=True
    )
    
bh

Graphics3d Object

In [15]:
sage.plot.plot3d.shapes.Sphere(4, color="red", viewer="threejs")

Graphics3d Object